# Kernel Density for Weekly and Monthly Discrete Data

In [1]:
import pandas as pd
import glob
import warnings
import arcpy
warnings.filterwarnings('ignore')
from arcpy import env
from arcpy.sa import *
arcpy.env.overwriteOutput = True
import os
import geopandas as gpd
import rasterio
import rasterio.mask
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import TwoSlopeNorm, LinearSegmentedColormap

In [2]:
gis_path = r'F:/SEACAR_WQ_2024/GIS_Data/'

## Create essential folders

In [4]:
kernel_week_path  = gis_path + "/raster_output/kernel_density_week"
kernel_month_path = gis_path + "/raster_output/kernel_density_month"
if not os.path.exists(kernel_week_path):
    os.makedirs(kernel_week_path)
    os.makedirs(kernel_week_path+"/shp")
    os.makedirs(kernel_week_path+"/kernel_density")
if not os.path.exists(kernel_month_path):
    os.makedirs(kernel_month_path)
    os.makedirs(kernel_month_path+"/shp")
    os.makedirs(kernel_month_path+"/kernel_density")

In [5]:
dfweek_diff = pd.read_csv(gis_path + 'week_difference.csv', low_memory=False)
dfweek_diff_drop = dfweek_diff.dropna(subset=["ConValue"]).reset_index(drop=True)
dfweek_diff_group = dfweek_diff_drop.groupby(["WaterBody","ParameterName","Latitude_DD","Longitude_DD",
                                              "x","y","WbodyAcronym","ParaAcronym"]).agg({"ResultValue":"mean","ConValue":"mean"}).reset_index()
dfweek_diff_group["Difference"] = dfweek_diff_group["ConValue"] - dfweek_diff_group["ResultValue"]
wb_list   = list(dfweek_diff_group.WbodyAcronym.unique())
para_list = list(dfweek_diff_group.ParaAcronym.unique())

## Define function to run the kernel density function

In [6]:
def kernel_density(workspace,df,search_radius):
    for wb in wb_list:
        for para in para_list:
            env.workspace = workspace
            name = str(wb)+"_"+str(para)
            search_radius = int(search_radius)
            df_use = df[(df["WbodyAcronym"]==str(wb))&(df["ParaAcronym"]==str(para))]
            df_use.to_csv(env.workspace+f"/shp/{name}.csv")
            arcpy.management.XYTableToPoint(env.workspace+f"/shp/{name}.csv", env.workspace+f"/shp/{name}.shp",
                                "x", "y", "Difference",arcpy.SpatialReference(3086))    
            print(name,arcpy.management.GetCount(env.workspace+f"/shp/{name}.shp"))
            outKDens = KernelDensity(in_features=env.workspace+f"/shp/{name}.shp", population_field="Difference",search_radius=search_radius)
            output_tif = env.workspace + f"/kernel_density/{name}_kd.tif"
            outKDens.save(output_tif)

## Generate kernel density rasters for weekly data
Here, the search_radius parameter is set as 5000

In [7]:
kernel_density(kernel_week_path,dfweek_diff_group,5000)

BB_DO_mgl 45
BB_Sal_ppt 30
BB_Turb_ntu 30
BB_T_c 47
CH_DO_mgl 388
CH_Sal_ppt 442
CH_Turb_ntu 37
CH_T_c 467
EB_DO_mgl 87
EB_Sal_ppt 19
EB_Turb_ntu 55
EB_T_c 104
GTM_DO_mgl 72
GTM_Sal_ppt 69
GTM_Turb_ntu 6
GTM_T_c 69


## Generate kernel density rasters for monthly data

In [8]:
dfmonth_diff = pd.read_csv(gis_path + 'month_difference.csv', low_memory=False)
dfmonth_diff_drop = dfmonth_diff.dropna(subset=["ConValue"]).reset_index(drop=True)
dfmonth_diff_group = dfmonth_diff_drop.groupby(["WaterBody","ParameterName","Latitude_DD","Longitude_DD",
                                                "x","y","WbodyAcronym","ParaAcronym"]).agg({"ResultValue":"mean","ConValue":"mean"}).reset_index()
dfmonth_diff_group["Difference"] = dfmonth_diff_group["ConValue"] - dfmonth_diff_group["ResultValue"]
wb_list   = list(dfmonth_diff_group.WbodyAcronym.unique())
para_list = list(dfmonth_diff_group.ParaAcronym.unique())

In [9]:
kernel_density(kernel_month_path,dfmonth_diff_group,5000)

BB_DO_mgl 45
BB_Sal_ppt 30
BB_Turb_ntu 30
BB_T_c 47
CH_DO_mgl 452
CH_Sal_ppt 428
CH_Turb_ntu 41
CH_T_c 453
EB_DO_mgl 99
EB_Sal_ppt 19
EB_Turb_ntu 81
EB_T_c 101
GTM_DO_mgl 71
GTM_Sal_ppt 69
GTM_Turb_ntu 6
GTM_T_c 69
